# DataFrame & Column

### Objectives
1. Construct columns
2. Subset columns
3. Add or replace columns
4. Subset rows
5. Sort rows

### Methods
- [DataFrame](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html): `select`, `selectExpr`, `drop`, `withColumn`, `withColumnRenamed`, `filter`, `distinct`, `limit`, `sort`
- [Column](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/column.html): `alias`, `isin`, `cast`, `isNotNull`, `desc`, operators


### Load BedBricks events dataset from inline CSV

In [ ]:
import pandas as pd

data = [{
    "user_id": "u1",
    "device": "Android",
    "event_timestamp": 1650000000000,
    "user_first_touch_timestamp": 1649990000000,
    "geo.city": "Raleigh",
    "geo.state": "NC",
    "traffic_source": "search",
    "ecommerce.purchase_revenue_in_usd": 199.99,
    "ecommerce.total_item_quantity": 2
}]

pdf = pd.DataFrame(data)
csv_path = "/dbfs/tmp/events_data.csv"
pdf.to_csv(csv_path, index=False)

events_df = (spark.read
             .option("header", "true")
             .option("inferSchema", "true")
             .csv(csv_path)
             .withColumnRenamed("geo.city", "geo_city")
             .withColumnRenamed("geo.state", "geo_state")
             .withColumnRenamed("ecommerce.purchase_revenue_in_usd", "ecom_revenue")
             .withColumnRenamed("ecommerce.total_item_quantity", "ecom_quantity"))

events_df.cache()
display(events_df)

### Column Expressions and Construction

In [ ]:
from pyspark.sql.functions import col

print(events_df.device)
print(events_df["device"])
print(col("device"))

In [ ]:
col("ecom_revenue") + col("ecom_quantity")
col("event_timestamp").desc()
(col("ecom_revenue") * 100).cast("int")

In [ ]:
rev_df = (events_df
         .filter(col("ecom_revenue").isNotNull())
         .withColumn("purchase_revenue", (col("ecom_revenue") * 100).cast("int"))
         .withColumn("avg_purchase_revenue", col("ecom_revenue") / col("ecom_quantity"))
         .sort(col("avg_purchase_revenue").desc()))
display(rev_df)

### DataFrame Transformations: select, drop, withColumn, etc.

In [ ]:
devices_df = events_df.select("user_id", "device")
display(devices_df)

In [ ]:
locations_df = events_df.select(
    "user_id", 
    col("geo_city").alias("city"), 
    col("geo_state").alias("state"))
display(locations_df)

In [ ]:
apple_df = events_df.selectExpr("user_id", "device in ('macOS', 'iOS') as apple_user")
display(apple_df)

In [ ]:
anonymous_df = events_df.drop("user_id", "geo_city", "device")
display(anonymous_df)

In [ ]:
mobile_df = events_df.withColumn("mobile", col("device").isin("iOS", "Android"))
display(mobile_df)

In [ ]:
location_df = events_df.withColumnRenamed("geo_city", "city")
display(location_df)

In [ ]:
purchases_df = events_df.filter(col("ecom_quantity") > 0)
display(purchases_df)

In [ ]:
distinct_users_df = events_df.dropDuplicates(["user_id"])
display(distinct_users_df)

In [ ]:
limit_df = events_df.limit(100)
display(limit_df)

In [ ]:
increase_timestamps_df = events_df.sort("event_timestamp")
display(increase_timestamps_df)

In [ ]:
decrease_sessions_df = events_df.sort(col("user_first_touch_timestamp").desc(), col("event_timestamp"))
display(decrease_sessions_df)